# Path Optimization (KOMO)



## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../ry')

from libry import *
from numpy import *

K = Config()
D = K.view()
K.addFile('../rai-robotModels/pr2/pr2.g')
K.addFile('../test/kitchen.g')

Let's add some objects

In [2]:
K.addFrame('item1', 'sink1', 'type:ssBox Q:<t(-.1 -.1 .52)> size:[.1 .1 .25 .02] color:[1. 0. 0.], contact' )
K.addFrame('item2', 'sink1', 'type:ssBox Q:<t(.1 .1 .52)> size:[.1 .1 .25 .02] color:[1. 1. 0.], contact' )
K.addFrame('tray', 'stove1', 'type:ssBox Q:<t(.0 .0 .42)> size:[.2 .2 .05 .02] color:[0. 1. 0.], contact' )

180

compute a collision free path to touch object item1 with pr2L:

In [3]:
X0 = K.getFrameState()

obj1 = "item1";
arm = "pr2R";

komo = K.komo_path(1.,20, 10.);

komo.addObjective([], OT.sos, FS.transAccelerations, [], [1.]);
komo.addObjective([], OT.eq, FS.accumulatedCollisions);
komo.addObjective([], OT.ineq, FS.jointLimits);
komo.addObjective([1.], OT.eq, FS.distance, [arm, obj1]);
komo.addObjective([.9,1.], OT.sos, FS.positionDiff, ["endeffWorkspace", obj1], [1e0]);
komo.addObjective(time=[1.], type=OT.eq, feature=FS.qItself, order=1);

komo.optimize();
komo.getReport()

[{'description': 'QuaternionNorms', 'sos_value': 0.0, 'type': 'sos'},
 {'feature': 'Transition', 'sos_value': 3.1548225930515295, 'type': 'sos'},
 {'eq_sumOfAbs': 0.00022084378362776172, 'feature': 'ProxyCost', 'type': 'eq'},
 {'feature': 'LimitsConstraint', 'type': 'ineq'},
 {'eq_sumOfAbs': 0.0010835191783960564,
  'feature': 'dist',
  'o1': 'pr2R',
  'o2': 'item1',
  'type': 'eq'},
 {'feature': 'posDiff',
  'o1': 'endeffWorkspace',
  'o2': 'item1',
  'sos_value': 0.5625589126906487,
  'type': 'sos'},
 {'description': 'qItself:ALL',
  'eq_sumOfAbs': 0.001475273949729114,
  'type': 'eq'}]

In [4]:
komo.displayTrajectory()

Best way to read out: grab a configuration into K and analyze it there:

In [5]:
K.setFrameState( komo.getConfiguration(19) )
K.getJointState()

array([-0.75705433,  1.2092785 ,  0.7422905 ,  0.10001675,  0.00238006,
       -0.00234903,  0.01862283,  1.14758456,  0.39312896,  0.39070544,
        0.41016126, -1.13796193,  0.93417891, -1.38239522, -2.04966149,
       -1.62043956,  1.50198354, -0.37057198, -0.49028672, -0.50052527,
        0.50073802,  0.09748649,  0.09623575,  0.00963027,  0.00973904])